# Weather Data Web Scraping
## Goal
The goal of this notebook is to scrape historical weather data for the 33 busiest airports in the US from Weather Underground. Two notes on this choice:
1. This data should be available from NOAA, but I could not get it to download.
2. Weather Underground does have an API, but it is not free so I had to resort to scraping the historical data table using selenium rather than a cleaner method.

## Brief Overview
The program is relatively simple: access the data for an airport in a given month and year, pull the data table containing temperature, wind speed, and pressure mins, maxes, and averages as well as the precipitation each day. Then cycle through every month from January 2021 through December 2025. While we do not have flight delay data from December 2025, it could prove useful when I update my model with newer flight data.

## Detailed Overview
A more detailed overview of this program is as follows. The outermost loop is through all the busiest airports followed by a first inner loop over 2021 through 2025 and finally an innermost loop over the months from January through December. In the innermost loop, we access the site of the form https://www.wunderground.com/history/monthly/KPDX/date/2025-1 where "KPDX" is replaced by "K" plus the 3 letter airport code with the exception of Honolulu, which uses PHNL. (I should note that no data was available for San Diego or Orlando. I will work on addressing these in a separate notebook, likely through weather averages compiled for each location.) From there, we look for the data table "observation-table" and locate the data we want in the table through "tr" and "td" tags. This gives us a list of lists (e.g. [['Sep'], ['1'], ['2'], ..., ['Max', 'Avg', 'Min'], ['44.0', '43.2', '42.1'], ...]) for all of our data of interest. We count the number of days first, because some data is incomplete (e.g. only 17 days of data in June 2022 for an airport), and then appropriately slice the data up into a dict and finally a dataframe. Dataframes from each airport each month are concatenated together until we have a full listing of the historical record across the date range of interest.

## Future Improvements
Moving forward, there are a few areas that could improve this data scraping method:
* Automatically handle regions with no data. San Diego and Orlando had to be adjusted by hand.
* Pull data from other sources (e.g. NOAA).
* Create a less burdensome means of recognizing when the data changes from dates to pressures to temperatures, etc.

In [31]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
import pandas as pd
import time
import os

In [59]:
# Some weather data is incomplete (e.g. only 17 days of data in March one year) so see how many days of data there are
def get_num_days(data):
    num_days = 0
    for item in data[1:]:
        try:
            temp = float(item[0])
            num_days += 1
        except ValueError:
            return num_days

    return num_days

# Convert the list of lists we scrape from Weather Underground and compile it into a dict then a dataframe
def data_to_df(data, cur_month, cur_year):
    days_in_month = get_num_days(data)

    data_dict = {}
    data_dict[all_data[0][0]] = [date[0] for date in data[1:1+days_in_month]]
    data_dict['Temp_' + data[1+days_in_month][0]] = [temp[0] for temp in data[2+days_in_month:2+days_in_month*2]]
    data_dict['Temp_' + data[1+days_in_month][1]] = [temp[1] for temp in data[2+days_in_month:2+days_in_month*2]]
    data_dict['Temp_' + data[1+days_in_month][2]] = [temp[2] for temp in data[2+days_in_month:2+days_in_month*2]]
    data_dict['WindSpeed_' + data[4+days_in_month*4][0]] = [windspeed[0] for windspeed in data[5+days_in_month*4:5+days_in_month*5]]
    data_dict['WindSpeed_' + data[4+days_in_month*4][1]] = [windspeed[1] for windspeed in data[5+days_in_month*4:5+days_in_month*5]]
    data_dict['WindSpeed_' + data[4+days_in_month*4][2]] = [windspeed[2] for windspeed in data[5+days_in_month*4:5+days_in_month*5]]
    data_dict['Pressure_' + data[5+days_in_month*5][0]] = [pressure[0] for pressure in data[6+days_in_month*5:6+days_in_month*6]]
    data_dict['Pressure_' + data[5+days_in_month*5][1]] = [pressure[1] for pressure in data[6+days_in_month*5:6+days_in_month*6]]
    data_dict['Pressure_' + data[5+days_in_month*5][2]] = [pressure[2] for pressure in data[6+days_in_month*5:6+days_in_month*6]]
    data_dict['Precipitation' + data[6+days_in_month*6][0]] = [precip[0] for precip in data[7+days_in_month*6:7+days_in_month*7]]

    df = pd.DataFrame(data_dict)
    df['Month'] = all_data[0][0]
    df['Day'] = df[all_data[0][0]]
    df['Year'] = cur_year
    df['Date'] = pd.to_datetime(df['Year'].astype(str) + '-' + df['Month'] + '-' + df['Day'].astype(str), format='%Y-%b-%d')
    df.drop(columns=[all_data[0][0], 'Month', 'Day', 'Year'], inplace=True)

    return df

In [68]:
# Busiest airports to loop through
busiest_airports = ['ATL', 'DFW', 'DEN', 'ORD', 'LAX', 'JFK', 'CLT', 'LAS', 'MCO',
                    'MIA', 'PHX', 'SEA', 'SFO', 'EWR', 'IAH', 'BOS', 'MSP', 'FLL',
                    'LGA', 'DTW', 'PHL', 'SLC', 'BWI', 'IAD', 'SAN', 'DCA', 'TPA',
                    'BNA', 'AUS', 'HNL', 'MDW', 'DAL', 'PDX']

# Loop through each airport and pull weather data from January 2021 through December 2025
for airport in busiest_airports:
    df = pd.DataFrame()
    
    for cur_year in range(2021, 2026):
        if cur_year == 2020 or cur_year == 2024:
            month_days['Feb'] = 29
        else:
            month_days['Feb'] = 28
            
        for cur_month in range(1, 13):
            # Honolulu uses a different URL than other airports for some reason
            if airport == 'HNL':
                url = 'https://www.wunderground.com/history/monthly/' + 'P' + airport + '/date/' + str(cur_year) + '-' + str(cur_month)
            else:
                url = 'https://www.wunderground.com/history/monthly/' + 'K' + airport + '/date/' + str(cur_year) + '-' + str(cur_month)

            # Use firefox with ublock origin to avoid issues with popups obscuring the table
            service = Service('C:/Users/dloso/Downloads/geckodriver-v0.36.0-win64/geckodriver.exe')
            driver = webdriver.Firefox(service=service)
            driver.install_addon('C:/Users/dloso/AppData/Roaming/Mozilla/Firefox/Profiles/mrrn081x.default-release-1735758974795/extensions/uBlock0@raymondhill.net.xpi')
            driver.get(url)
            
            try:
                # Wait for the specific observation table container to load
                wait = WebDriverWait(driver, 10)
                table_container = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "observation-table")))
            
                # Extract headers (Top Level and Subheaders)
                # WU often uses 'thead' or specific 'tr' classes for these
                header_rows = table_container.find_elements(By.TAG_NAME, "tr")[:2]
                
                # Extract all data rows
                data_rows = table_container.find_elements(By.TAG_NAME, "tr")[2:]

                # String out data into a list of lists
                all_data = []
                for row in data_rows:
                    cols = row.find_elements(By.TAG_NAME, "td")
                    all_data.append([c.text for c in cols])

                # Convert to a dataframe and concatenate with previous data for this airport
                df_cur = data_to_df(all_data, cur_month, cur_year)
                df = pd.concat([df, df_cur])
            finally:
                driver.quit()
                print(f'{airport} {all_data[0][0]} {cur_year} complete.')
    
    # Write to file
    df.to_csv('data/weather/' + airport + '_Weather.csv')
    print('Wrote data to data/weather/' + airport + '_Weather.csv')

HNL Jan 2021 complete.
HNL Feb 2021 complete.
HNL Mar 2021 complete.
HNL Apr 2021 complete.
HNL May 2021 complete.
HNL Jun 2021 complete.
HNL Jul 2021 complete.
HNL Aug 2021 complete.
HNL Sep 2021 complete.
HNL Oct 2021 complete.
HNL Nov 2021 complete.
HNL Dec 2021 complete.
HNL Jan 2022 complete.
HNL Feb 2022 complete.
HNL Mar 2022 complete.
HNL Apr 2022 complete.
HNL May 2022 complete.
HNL Jun 2022 complete.
HNL Jul 2022 complete.
HNL Aug 2022 complete.
HNL Sep 2022 complete.
HNL Oct 2022 complete.
HNL Nov 2022 complete.
HNL Dec 2022 complete.
HNL Jan 2023 complete.
HNL Feb 2023 complete.
HNL Mar 2023 complete.
HNL Apr 2023 complete.
HNL May 2023 complete.
HNL Jun 2023 complete.
HNL Jul 2023 complete.
HNL Aug 2023 complete.
HNL Sep 2023 complete.
HNL Oct 2023 complete.
HNL Nov 2023 complete.
HNL Dec 2023 complete.
HNL Jan 2024 complete.
HNL Feb 2024 complete.
HNL Mar 2024 complete.
HNL Apr 2024 complete.
HNL May 2024 complete.
HNL Jun 2024 complete.
HNL Jul 2024 complete.
HNL Aug 202